In [1]:
import pandas as pd
import os
import json
import pandas as pd
import os
import json
import glob


# Define the path to the data folder, starting from the current working directory
current_dir = os.path.abspath('')
project_root = os.path.join(current_dir, os.pardir)  # Move one level up to the project root
data_folder = os.path.normpath(os.path.join(project_root, 'root', 'dags', 'data'))

print("Data folder path:", data_folder)

# Check if the data folder exists
if os.path.exists(data_folder):
    print("Data folder exists.")
else:
    print("Data folder does not exist.")


Data folder path: c:\Users\Vicente Renart\Desktop\bees_data_pipeline\root\dags\data
Data folder exists.


In [2]:
# Bronze Layer
bronze_path = os.path.join(data_folder, "bronze_breweries.json")
try:
    with open(bronze_path, 'r') as file:
        bronze_data = json.load(file)
        bronze_df = pd.DataFrame(bronze_data)
        print("Bronze Layer Data:")
        print(bronze_df.head())
except Exception as e:
    print("Error loading Bronze layer data:", e)

Bronze Layer Data:
                                     id                     name brewery_type  \
0  5128df48-79fc-4f0f-8b52-d06be54d0cec         (405) Brewing Co        micro   
1  9c5a66c8-cc13-416f-a5d9-0a769c87d318         (512) Brewing Co        micro   
2  34e8c68b-6146-453f-a4b9-1f6cd99a5ada  1 of Us Brewing Company        micro   
3  ef970757-fe42-416f-931d-722451f1f59c     10 Barrel Brewing Co        large   
4  6d14b220-8926-4521-8d19-b98a2d6ec3db     10 Barrel Brewing Co        large   

               address_1 address_2 address_3            city state_province  \
0         1716 Topeka St      None      None          Norman       Oklahoma   
1  407 Radam Ln Ste F200      None      None          Austin          Texas   
2    8100 Washington Ave      None      None  Mount Pleasant      Wisconsin   
3              1501 E St      None      None       San Diego     California   
4          62970 18th St      None      None            Bend         Oregon   

  postal_code      

In [ ]:
# Function to load and compare data between Bronze and Silver layers
def validate_state_data(selected_state):
    # Define project directories for Bronze and Silver layers
    current_dir = os.path.abspath('')
    project_root = os.path.join(current_dir, os.pardir)
    data_folder = os.path.normpath(os.path.join(project_root, 'root', 'dags', 'data'))
    
    bronze_file_path = os.path.join(data_folder, 'bronze_breweries.json')
    silver_file_path = os.path.join(data_folder, 'silver_breweries', f"{selected_state}.parquet")
    
    # Check if both files exist
    if not os.path.exists(bronze_file_path):
        print(f"Bronze data file does not exist.")
        return
    if not os.path.exists(silver_file_path):
        print(f"Silver file for state '{selected_state}' does not exist.")
        return

    try:
        # Load Bronze and Silver data
        df_bronze = pd.read_json(bronze_file_path)
        df_silver = pd.read_parquet(silver_file_path)

        # Filter Bronze data for the selected state
        df_bronze_state = df_bronze[df_bronze['state'] == selected_state]
        
        # Display sample rows from both datasets for comparison
        print(f"Bronze Layer Data for state '{selected_state}':")
        print(df_bronze_state.head())
        
        print(f"\nSilver Layer Data for state '{selected_state}':")
        print(df_silver.head())
        
        # Validate that the Silver data contains only the selected state
        silver_states = df_silver['state'].unique()
        
        if selected_state in silver_states and len(silver_states) == 1:
            print(f"\nValidation passed: Data in Silver Layer for '{selected_state}' is consistent with Bronze Layer.")
        else:
            print(f"\nValidation failed: Silver Layer data for '{selected_state}' contains data from other states or is incomplete.")
    
    except Exception as e:
        print(f"Error comparing data for state '{selected_state}':", e)

# Example usage
validate_state_data("Arizona")  # Change the state here to validate a different one

# Possible states: Arizona, California, Colorado, Delaware, Idaho, Illinois, Indiana, Iowa, Laois, Maryland, 
# Massachusetts, Michigan, Minnesota, Mississippi, Nevada, New York, North Carolina, Ohio, Oklahoma, Oregon, 
# Pennsylvania, South Carolina, Texas, Vermont, Virginia, Washington, Wisconsin



Bronze Layer Data for state 'Arizona':
                                      id  \
20  fb94830f-6196-4f59-9189-c9060b778085   
21  0faa0fb2-fffa-416d-9eab-46f67477c8ef   
48  4f4b5b34-d572-4dff-a18f-47e507c073e6   

                                             name brewery_type  \
20                        12 West Brewing Company        micro   
21  12 West Brewing Company - Production Facility        micro   
48                                   1912 Brewing        micro   

                     address_1 address_2  address_3     city state_province  \
20        3000 E Ray Rd Bldg 6      None        NaN  Gilbert        Arizona   
21                        None      None        NaN     Mesa        Arizona   
48  2045 N Forbes Blvd Ste 105      None        NaN   Tucson        Arizona   

   postal_code        country   longitude   latitude         phone  \
20  85296-7832  United States         NaN        NaN  6.023395e+09   
21       85207  United States -111.586066  33.436188          

In [12]:
#usar a mesma logica da silver para a gold. fazer o select desses dados para ver se ta mostrando a mesma informação

# Gold Layer (Parquet File)
gold_path = os.path.join(data_folder, "gold_breweries_aggregated.parquet")

# Check if the Gold Layer file exists
if not os.path.exists(gold_path):
    print(f"Gold layer file '{gold_path}' does not exist. Please run the aggregation step first.")
else:
    try:
        # Load the Gold Layer data
        gold_df = pd.read_parquet(gold_path)
        print("Gold Layer Data:")
        print(gold_df.head(1000))
    except Exception as e:
        print("Error loading Gold layer data:", e)


Gold Layer Data:
   brewery_type           state  brewery_count
0       brewpub        Colorado              1
1       brewpub        New York              2
2       brewpub  North Carolina              1
3       brewpub            Ohio              1
4       brewpub          Oregon              2
5       brewpub  South Carolina              1
6        closed      California              1
7        closed      Washington              1
8      contract        Maryland              1
9         large      California              1
10        large        Colorado              1
11        large           Idaho              1
12        large          Oregon              4
13        micro         Arizona              3
14        micro      California              2
15        micro        Colorado              1
16        micro        Delaware              1
17        micro        Illinois              1
18        micro         Indiana              3
19        micro            Iowa            